In [6]:
import mlflow
import os

TRACKING_SERVER_HOST = os.environ.get("TRACKING_SERVER_HOST")

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [8]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://52.14.126.221:5000'


In [10]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-manual-artifacts-remote/0', creation_time=1715819053050, experiment_id='0', last_update_time=1715819053050, lifecycle_stage='active', name='Default', tags={}>]

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/05/16 00:56:00 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
/home/codespace/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


default artifacts URI: 's3://mlflow-manual-artifacts-remote/1/f09f8e8dd614499491afd27bd97e2017/artifacts'


In [12]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-manual-artifacts-remote/1', creation_time=1715820960446, experiment_id='1', last_update_time=1715820960446, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-manual-artifacts-remote/0', creation_time=1715819053050, experiment_id='0', last_update_time=1715819053050, lifecycle_stage='active', name='Default', tags={}>]

In [13]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [15]:
client.search_registered_models()

[]

In [20]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/05/16 01:02:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1715821320433, current_stage='None', description='', last_updated_timestamp=1715821320433, name='iris-classifier', run_id='f09f8e8dd614499491afd27bd97e2017', run_link='', source='s3://mlflow-manual-artifacts-remote/1/f09f8e8dd614499491afd27bd97e2017/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [21]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1715821320357, description='', last_updated_timestamp=1715821320433, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1715821320433, current_stage='None', description='', last_updated_timestamp=1715821320433, name='iris-classifier', run_id='f09f8e8dd614499491afd27bd97e2017', run_link='', source='s3://mlflow-manual-artifacts-remote/1/f09f8e8dd614499491afd27bd97e2017/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]